In [1]:
import numpy as np 
import pandas as pd

In [2]:
from utils import *
from preprocessing import *

In [3]:
# load data using utf8 format

data = pd.read_csv('./datasets/public_train.csv', encoding='utf-8')

In [4]:
# fill 0 to num_like_post, num_commnet_post, num_share_post
data.num_like_post.fillna(0, inplace=True)
data.num_comment_post.fillna(0, inplace=True)
data.num_share_post.fillna(0, inplace=True)

# check again
check_like = data.num_like_post.isna().sum()
check_commnet = data.num_comment_post.isna().sum()
check_share = data.num_share_post.isna().sum()

print(f'num_like_post has nan {check_like != 0}')
print(f'num_comment_post has nan {check_commnet != 0}')
print(f'num_share_post has nan {check_share != 0}')

num_like_post has nan False
num_comment_post has nan False
num_share_post has nan False


In [5]:
# drop id, user_name, timestamp_post columns
data = data.drop(['user_name', 'timestamp_post'], axis=1).reset_index(drop=True)

# view data
data

,id,post_message,num_like_post,num_comment_post,num_share_post,label
0,1,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0
1,2,<URL>,11,5,3,0
2,3,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0
3,4,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0
4,5,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0
...,...,...,...,...,...,...
4367,4368,BÀ MẸ VIỆT NAM ANH HÙNG 95 TUỔI MAY KHẨU TRANG...,5800,1300,12000,0
4368,4369,Nguồn cung khan kiếm nhưng nhu cầu cao tạo áp ...,21,1,0,0
4369,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,3,1,0,1
4370,4371,Đến bây giờ mới biết chỉ cần học lái xe hạng B...,144,38,87,1


In [6]:
# drop duplicate samples has <URL>, [<URL>](<URL>)
char_drop = ['<URL>', '[<URL>](<URL>)']
index = []
for char in char_drop:
    index.extend(list(data[data.post_message == char].index))
data = data.drop(index, axis=0).reset_index(drop=True)

# view data
data

,id,post_message,num_like_post,num_comment_post,num_share_post,label
0,1,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0
1,3,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0
2,4,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0
3,5,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0
4,6,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0
...,...,...,...,...,...,...
4307,4368,BÀ MẸ VIỆT NAM ANH HÙNG 95 TUỔI MAY KHẨU TRANG...,5800,1300,12000,0
4308,4369,Nguồn cung khan kiếm nhưng nhu cầu cao tạo áp ...,21,1,0,0
4309,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,3,1,0,1
4310,4371,Đến bây giờ mới biết chỉ cần học lái xe hạng B...,144,38,87,1


In [7]:
#check info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4312 non-null   int64 
 1   post_message      4311 non-null   object
 2   num_like_post     4312 non-null   object
 3   num_comment_post  4312 non-null   object
 4   num_share_post    4312 non-null   object
 5   label             4312 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 202.2+ KB


In [8]:
index = []
for i in range(len(data)):
    try:
        int(float(data.num_comment_post[i]))
        int(float(data.num_like_post[i]))
        int(float(data.num_share_post[i]))
    except:
        index.append(i)
data_temp = data.iloc[index,:]

In [9]:
data = data.drop(index=index, axis=0).reset_index(drop=True)
data

,id,post_message,num_like_post,num_comment_post,num_share_post,label
0,1,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0
1,3,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0
2,4,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0
3,5,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0
4,6,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0
...,...,...,...,...,...,...
4220,4368,BÀ MẸ VIỆT NAM ANH HÙNG 95 TUỔI MAY KHẨU TRANG...,5800,1300,12000,0
4221,4369,Nguồn cung khan kiếm nhưng nhu cầu cao tạo áp ...,21,1,0,0
4222,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,3,1,0,1
4223,4371,Đến bây giờ mới biết chỉ cần học lái xe hạng B...,144,38,87,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4225 entries, 0 to 4224
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4225 non-null   int64 
 1   post_message      4224 non-null   object
 2   num_like_post     4225 non-null   object
 3   num_comment_post  4225 non-null   object
 4   num_share_post    4225 non-null   object
 5   label             4225 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 198.2+ KB


In [11]:
# convert post_message, num_like_post, num_comment_post, num_share_post to int64
columns = ['num_like_post', 'num_comment_post', 'num_share_post']

data[columns] = data[columns].apply(pd.to_numeric).astype('int64')

data

,id,post_message,num_like_post,num_comment_post,num_share_post,label
0,1,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173,0
1,3,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19,0
2,4,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0,0
3,5,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54,0
4,6,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0
...,...,...,...,...,...,...
4220,4368,BÀ MẸ VIỆT NAM ANH HÙNG 95 TUỔI MAY KHẨU TRANG...,5800,1300,12000,0
4221,4369,Nguồn cung khan kiếm nhưng nhu cầu cao tạo áp ...,21,1,0,0
4222,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,3,1,0,1
4223,4371,Đến bây giờ mới biết chỉ cần học lái xe hạng B...,144,38,87,1


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4225 entries, 0 to 4224
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4225 non-null   int64 
 1   post_message      4224 non-null   object
 2   num_like_post     4225 non-null   int64 
 3   num_comment_post  4225 non-null   int64 
 4   num_share_post    4225 non-null   int64 
 5   label             4225 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 198.2+ KB


In [13]:
nul_id = data[data.post_message.isnull()].index
nul_id

Int64Index([2186], dtype='int64')

In [14]:
data.iloc[nul_id,:]

,id,post_message,num_like_post,num_comment_post,num_share_post,label
2186,2248,NaN,9717,28,88,0


In [15]:
data = data.drop(index=[2186], axis=0).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4224 entries, 0 to 4223
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4224 non-null   int64 
 1   post_message      4224 non-null   object
 2   num_like_post     4224 non-null   int64 
 3   num_comment_post  4224 non-null   int64 
 4   num_share_post    4224 non-null   int64 
 5   label             4224 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 198.1+ KB


In [16]:
cols = ['id','num_like_post', 'num_comment_post', 'num_share_post', 'label']

# data[cols].to_csv('num_data_del.csv',index=False, encoding='utf-8')

In [17]:
cols = ['id', 'post_message', 'label']

# data[cols].to_csv('cat_data_del.csv', index=False, encoding='utf-8')

In [18]:
# down samples for num_data
num_data = pd.read_csv('num_data_del.csv', encoding='utf-8')

index_0 = list(num_data[num_data.label == 0].index)
index_1 = list(num_data[num_data.label == 1].index)

ids = list(np.random.choice(index_0, size=800, replace=False))

index = ids + index_1

# num_data.iloc[index,:].reset_index(drop=True).to_csv('num_data_del_down.csv', encoding='utf-8', index=False)

In [19]:
# down samples for cat_data
cat_data = pd.read_csv('cat_data_del.csv', encoding='utf-8')

index_0 = list(num_data[num_data.label == 0].index)
index_1 = list(num_data[num_data.label == 1].index)

ids = list(np.random.choice(index_0, size=800, replace=False))

index = ids + index_1

# cat_data.iloc[index,:].reset_index(drop=True).to_csv('cat_data_del_down.csv', encoding='utf-8', index=False)

In [20]:
cat_data = pd.read_csv('cat_data_del.csv', encoding='utf8')

cat_data

,id,post_message,label
0,1,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",0
1,3,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,0
2,4,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,0
3,5,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,0
4,6,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,0
...,...,...,...
4219,4368,BÀ MẸ VIỆT NAM ANH HÙNG 95 TUỔI MAY KHẨU TRANG...,0
4220,4369,Nguồn cung khan kiếm nhưng nhu cầu cao tạo áp ...,0
4221,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,1
4222,4371,Đến bây giờ mới biết chỉ cần học lái xe hạng B...,1


In [21]:
from vncorenlp import VnCoreNLP

stopwords_file = "vietnamese-stopwords/vietnamese-stopwords.txt"

stopwords = get_stop_words(stopwords_file)

annotator = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

cat_data['post_message_preproced'] = cat_data.post_message.apply(text_preprocessing, stopwords= stopwords, annotator= annotator)

cat_data

,id,post_message,label,post_message_preproced
0,1,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",0,thăng cấp_bậc hàm cán_bộ chiến_sỹ hy_sinh đà_n...
1,3,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,0,tư_vấn mùa thi nộp hồ_sơ trúng_tuyển chương_tr...
2,4,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,0,cơ_quan cạnh_tranh thị_trường quyết_định điều_...
3,5,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,0,ca quảng_nam hành_khách chuyến bay bệnh_nhân n...
4,6,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,0,học thể dụ c thầy_giáo nguyễn_văn quân phụ trá...
...,...,...,...,...
4219,4368,BÀ MẸ VIỆT NAM ANH HÙNG 95 TUỔI MAY KHẨU TRANG...,0,mẹ việt_nam anh_hùng may khẩu_trang chống dịch...
4220,4369,Nguồn cung khan kiếm nhưng nhu cầu cao tạo áp ...,0,cung khan kiếm nhu_cầu áp_lực tăng_giá dự_án s...
4221,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,1,cảnh_tỉnh thanh_niên dân tổtốc độ bàn_thờ emoj...
4222,4371,Đến bây giờ mới biết chỉ cần học lái xe hạng B...,1,học lái_xe hạng chữa bá bệnh lãnh_đạo_tế xứ th...


In [22]:
cat_data.to_csv('cat_data_del_pre.csv', encoding='utf8', index=False)

In [23]:
num_del = pd.read_csv('num_data_del.csv', encoding='utf8')

num_del

,id,num_like_post,num_comment_post,num_share_post,label
0,1,19477,378,173,0
1,3,48,5,19,0
2,4,3,0,0,0
3,5,775,0,54,0
4,6,2,1,0,0
...,...,...,...,...,...
4219,4368,5800,1300,12000,0
4220,4369,21,1,0,0
4221,4370,3,1,0,1
4222,4371,144,38,87,1


In [24]:
num_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4224 entries, 0 to 4223
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                4224 non-null   int64
 1   num_like_post     4224 non-null   int64
 2   num_comment_post  4224 non-null   int64
 3   num_share_post    4224 non-null   int64
 4   label             4224 non-null   int64
dtypes: int64(5)
memory usage: 165.1 KB


In [25]:
num_del_0 = num_del[num_del.label == 0]
num_del_1 = num_del[num_del.label == 1]

num_del_0_test = num_del_0.sample(frac=0.2, replace=False)
num_del_1_test = num_del_1.sample(frac=0.2, replace=False)

num_test = pd.concat([num_del_1_test, num_del_0_test], ignore_index=True, axis=0)

num_test

,id,num_like_post,num_comment_post,num_share_post,label
0,2225,160,13,60,1
1,4093,2,3,0,1
2,4068,7,1,0,1
3,4067,143,4,21,1
4,2535,4,0,0,1
...,...,...,...,...,...
840,2806,1,0,0,0
841,2619,151,1,4,0
842,928,35,2,1,0
843,1090,278,44,34,0


In [26]:
# num_test.to_csv('num_test.csv', encoding='utf8', index=False)

In [28]:
cat_del = pd.read_csv('cat_data_del_pre.csv', encoding='utf8')
cat_test = cat_del.set_index('id').loc[num_test.id.values]

cat_test

,post_message,label,post_message_preproced
id,,,
2225,Dữ liệu mở của <URL> cho thấy mức độ đioxit l...,1,dữ_liệu url mức_độ đioxit lưu_huỳnh báo_động t...
4093,Tại sao cấm và cấm bằng cách nào?\nBộ Lao động...,1,cấm cấm lao_động thương_binh xã_hội cục trưởng...
4068,"Tạp chí Tài chính công khai cổ vũ, PR cho hoạt...",1,tạp_chí tài_chính công_khai cổ_vũ pr hoạt_động...
4067,Chợ đầu mối Bình Điền đóng cửa trong mùa dịch ...,1,chợ đầu_mối bình điền đóng_cửa mùa dịch covid ...
2535,"Thêm 45.000 ca mới 1 ngày, Mỹ vẫn căng nhể. Nh...",1,ca mỹ căng nhể bang california texas florida g...
...,...,...,...
2806,"Tiết kiệm được hơn 1 tỷ đồng, tôi muốn mua mản...",0,tiết_kiệm tỷ đồng mua mảnh đất vay_mượn xây vợ...
2619,HDBank tiếp tục dành thêm 10.000 tỷ đồng hỗ tr...,0,hdbank tỷ đồng khách_hàng doanh_nghiệp siêu hộ...
928,"Ngày xưa ""tam đảo"" 3 Tổng giám đốc công ty thà...",0,tam_đảo tổng_giám_đốc công_ty thành_viên fpt m...


In [29]:
cat_test = cat_test.reset_index()

cat_test

,id,post_message,label,post_message_preproced
0,2225,Dữ liệu mở của <URL> cho thấy mức độ đioxit l...,1,dữ_liệu url mức_độ đioxit lưu_huỳnh báo_động t...
1,4093,Tại sao cấm và cấm bằng cách nào?\nBộ Lao động...,1,cấm cấm lao_động thương_binh xã_hội cục trưởng...
2,4068,"Tạp chí Tài chính công khai cổ vũ, PR cho hoạt...",1,tạp_chí tài_chính công_khai cổ_vũ pr hoạt_động...
3,4067,Chợ đầu mối Bình Điền đóng cửa trong mùa dịch ...,1,chợ đầu_mối bình điền đóng_cửa mùa dịch covid ...
4,2535,"Thêm 45.000 ca mới 1 ngày, Mỹ vẫn căng nhể. Nh...",1,ca mỹ căng nhể bang california texas florida g...
...,...,...,...,...
840,2806,"Tiết kiệm được hơn 1 tỷ đồng, tôi muốn mua mản...",0,tiết_kiệm tỷ đồng mua mảnh đất vay_mượn xây vợ...
841,2619,HDBank tiếp tục dành thêm 10.000 tỷ đồng hỗ tr...,0,hdbank tỷ đồng khách_hàng doanh_nghiệp siêu hộ...
842,928,"Ngày xưa ""tam đảo"" 3 Tổng giám đốc công ty thà...",0,tam_đảo tổng_giám_đốc công_ty thành_viên fpt m...
843,1090,"GƯƠNG MẶT SHOWBIZ | ĐẠT G, DU UYÊN TRỞ LẠI VỚI...",0,gương_mặt showbiz du uyên trở_lại gói xôi vội ...


In [30]:
cat_test.to_csv('cat_test.csv', encoding='utf8', index=False)